# Exploring Nos Talents No Emplois
https://www.apec.fr/candidat/recherche-emploi.html/emploi

Imports

In [ ]:
import requests
import json
import time

import os

In [3]:
os.chdir('/Users/marinnagy/Documents/GitHub/NaturalJob/APEC/')

### Explore internal API

Base url and keyword

In [4]:
base_url = "https://www.apec.fr/cms/webservices/rechercheOffre"
keyword = "data"

Default payload

In [ ]:
payload = {
    "lieux": [],
    "fonctions": [],
    "statutPoste": [],
    "typesContrat": [],
    "typesConvention": ["143684", "143685", "143686", "143687", "143706"],
    "niveauxExperience": [],
    "idsEtablissement": [],
    "secteursActivite": [],
    "typesTeletravail": [],
    "idNomZonesDeplacement": [],
    "positionNumbersExcluded": [],
    "typeClient": "CADRE",
    "sorts": [{"type": "SCORE", "direction": "DESCENDING"}],
    "pagination": {"range": 20, "startIndex": 0},
    "activeFiltre": True,
    "pointGeolocDeReference": {"distance": 0},
    "motsCles": "data",
}

Test requests

In [13]:
headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

response = requests.post(base_url, json=payload, headers=headers)
print(response.status_code)
print(response.text)

200
{"resultats":[{"id":177625412,"numeroOffre":"177625412W","intitule":"Data Analyst/ Data Engineer F/H","intituleSurbrillance":"Data Analyst/ Data Engineer F/H","nomCommercial":"ICSIS NORD","lieuTexte":"Lille - 59","salaireTexte":"A partir de 35 k€ brut annuel","texteOffre":"Dans le cadre du développement de notre agence de Lille, nous recrutons actuellement des Data Analyst / Data Engineer.  La mission consiste à :  - Maîtrise de la donnée et garant de la qualité de son utilisation  - Analyser les besoins des utilisateurs et proposer des solutions i...","urlLogo":"/media_entreprise/716123/logo_ICSIS_NORD_716123.jpg","dateValidation":"2025-11-17T09:27:55.000+0000","datePublication":"2025-11-17T09:27:55.000+0000","indicateurOqa":true,"latitude":"50.6318417","longitude":"3.0469999","localisable":true,"score":53.194374,"offreConfidentielle":false,"secteurActivite":101569,"secteurActiviteParent":101753,"clientReel":true,"contractDuration":0,"typeContrat":101888,"origineCode":101859,"idNo

First try. There is 7500 offers to parse, lets try to increase the number of result per page

In [15]:
payload['pagination']['range'] = 100

headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

response = requests.post(base_url, json=payload, headers=headers)
print(response.status_code, flush=True)

content = response.json()
print('Nombre de résultat', len(content['resultats']))

200
Nombre de résultat 100


## Parsing the whole website

In [30]:
size = 100

headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

payload = {
    "lieux": [],
    "fonctions": [],
    "statutPoste": [],
    "typesContrat": [],
    "typesConvention": ["143684", "143685", "143686", "143687", "143706"],
    "niveauxExperience": [],
    "idsEtablissement": [],
    "secteursActivite": [],
    "typesTeletravail": [],
    "idNomZonesDeplacement": [],
    "positionNumbersExcluded": [],
    "typeClient": "CADRE",
    "sorts": [{"type": "SCORE", "direction": "DESCENDING"}],
    "pagination": {"range": size, "startIndex": 0},
    "activeFiltre": True,
    "pointGeolocDeReference": {"distance": 0},
    "motsCles": "data",
}

results = []

while True:
    response = requests.post(base_url, json=payload, headers=headers)
    content = response.json()
    if len(content['resultats']) == 0:
        break
    results.extend(content['resultats'])
    print(f"Fetched page {payload['pagination']['startIndex']/size} with {len(content['resultats'])} results")
    payload['pagination']['startIndex'] += size

Fetched page 0.0 with 100 results
Fetched page 1.0 with 100 results
Fetched page 2.0 with 100 results
Fetched page 3.0 with 100 results
Fetched page 4.0 with 100 results
Fetched page 5.0 with 100 results
Fetched page 6.0 with 100 results
Fetched page 7.0 with 100 results
Fetched page 8.0 with 100 results
Fetched page 9.0 with 100 results
Fetched page 10.0 with 100 results
Fetched page 11.0 with 100 results
Fetched page 12.0 with 100 results
Fetched page 13.0 with 100 results
Fetched page 14.0 with 100 results
Fetched page 15.0 with 100 results
Fetched page 16.0 with 100 results
Fetched page 17.0 with 100 results
Fetched page 18.0 with 100 results
Fetched page 19.0 with 100 results
Fetched page 20.0 with 100 results
Fetched page 21.0 with 100 results
Fetched page 22.0 with 100 results
Fetched page 23.0 with 100 results
Fetched page 24.0 with 100 results
Fetched page 25.0 with 100 results
Fetched page 26.0 with 100 results
Fetched page 27.0 with 100 results
Fetched page 28.0 with 100 res

Save it

In [34]:
print(f"Total results fetched: {len(results)}")
with open("response_APEC.json", "w") as f:
    json.dump(results, f, indent=3)

Total results fetched: 7528


## Complete offer descriptions
The offer description are truncated, lets parse each offer page to collect complete description

In [7]:
offer_base_url = "https://www.apec.fr/cms/webservices/offre/public"

headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

def safe_requests(url: str, params: dict, headers: dict, tic=0) -> dict:
    try:
        response = requests.get(url, params=params, headers=headers)
        content = response.json()
    except Exception as e:
        if tic > 2:
            print(f'Error requesting {params["numeroOffre"]}: {e}')
            return None
        time.sleep(.5)
        content = safe_requests(url, params, headers, tic=tic+1)
    return content

offers = []

for i, offer in enumerate(results):
    params = {'numeroOffre' : offer['numeroOffre']}
    content = safe_requests(offer_base_url, params=params, headers=headers)
    if content: 
        offers.append(content)
        print(f"Fetched offer {offer['numeroOffre']} ({i}/{len(results)})")

Fetched offer 177625412W (0/7528)
Fetched offer 177682861W (1/7528)
Fetched offer 177589950W (2/7528)
Fetched offer 177721754W (3/7528)
Fetched offer 177695684W (4/7528)
Fetched offer 177608505W (5/7528)
Fetched offer 177367778W (6/7528)
Fetched offer 177148521W (7/7528)
Fetched offer 177611906W (8/7528)
Fetched offer 177631987W (9/7528)
Fetched offer 177532194W (10/7528)
Fetched offer 177367617W (11/7528)
Fetched offer 177608595W (12/7528)
Fetched offer 177716105W (13/7528)
Fetched offer 177674265W (14/7528)
Fetched offer 177718828W (15/7528)
Fetched offer 177580400W (16/7528)
Fetched offer 177649298W (17/7528)
Fetched offer 177418205W (18/7528)
Fetched offer 177613168W (19/7528)
Fetched offer 177683125W (20/7528)
Fetched offer 177687337W (21/7528)
Fetched offer 177702125W (22/7528)
Fetched offer 177661555W (23/7528)
Fetched offer 177715532W (24/7528)
Fetched offer 177531500W (25/7528)
Fetched offer 177642301W (26/7528)
Fetched offer 177625907W (27/7528)
Fetched offer 177651284W (28/7

Save it

In [10]:
print(f"Total results fetched: {len(offers)}")
with open("complete_APEC.json", "w") as f:
    json.dump(offers, f, indent=3)

Total results fetched: 7526
